In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


In [2]:
df = sns.load_dataset("titanic")
df.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
features = ["sex", "age", "pclass", "fare"]
target = "survived"

df = df[features + [target]].dropna()


In [ ]:
X = df[features]
y = df[target]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)


In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
model = LogisticRegression()
model.fit(X_train_scaled, y_train)


In [ ]:
proba = model.predict_proba(X_test_scaled)[:, 1]
pred = model.predict(X_test_scaled)


In [ ]:
coef_df = pd.DataFrame({
    "feature": features,
    "coef": model.coef_[0]
})

coef_df


In [ ]:
coef_df["odds_multiplier"] = np.exp(coef_df["coef"])
coef_df


In [ ]:
passenger = pd.DataFrame({
    "sex": [1],
    "age": [30],
    "pclass": [3],
    "fare": [10]
})

passenger_scaled = scaler.transform(passenger)

probability = model.predict_proba(passenger_scaled)[0, 1]
probability


Модель має не запам’ятовувати дані, а узагальнювати закономірності


##### Underfitting (недонавчання)
Модель занадто проста і не вловлює закономірності

**Ознаки:**
> погано працює і на train, і на test \
> низьки метрики 

**Опрацювати:**
> ускладнити модель \
> додати ознаки \
> прибрати занадто сильну регуляризацію


##### Overfitting (перенавчання)
Модель запам’ятала train, але не працює на нових даних

> train: дуже добре
> test: погано

**Приклад:** Decision Tree без обмежень модель, яка вивчила напам’ять шум


**Опрацювати:**
> обмежити складність \
> більше даних \
> регуляризація \
> крос-валідація

**Варіант 1**
> тільки sex \
> accuracy = 65% і на train, і на test


**Варіант 2**
> дерево без обмежень \
> train = 99%, test = 70%


**Варіант 3**
> train = 78%, test = 76%

Linear Regression/Лінійна регресія прогнозує **ЧИСЛО**

**Приклади:**
> ціна квартири \
> час доставки \
> дохід користувача

Модель будує пряму лінію:
y = w(0) + w(1)x(1) + w(2)(x2) + ....

> w(0) - базове значення \
> w(i) — наскільки зміниться результат, якщо ознака зросте на 1

Приклад:price = 1000 * area + 5000



In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
data = fetch_california_housing(as_frame=True)
df = data.frame

df.head()


In [ ]:
target = "MedHouseVal" 
X = df[["MedInc"]] 
y = df[target]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42
)


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)


In [ ]:
model.coef_, model.intercept_


In [10]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    "n_estimators": [100, 200, 250, 300],
    "max_depth": [None, 5, 10],
    "min_samples_leaf": [1, 3, 5, 7, 10]
}

grid = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1
)

grid.fit(X_train, y_train)

grid.best_params_


{'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 200}

In [11]:
best_rf = grid.best_estimator_

pred_best = best_rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred_best))

Accuracy: 0.8044692737430168
